# DB2-Salesforce connector: Extract Bio information

In [1]:
# Parameters
hours_range = 24*365*10 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",  
} 

In [2]:
import sys
#sys.path.append(lib_dir)

import pandas as pd
import datetime

## Obtain all bios from DB2

In [3]:
## Extract all non-empty user bios
# Hourly update for new registrations
sql_query = "select user_id, profile_value from jos_user_profiles where profile_key = 'bio';"

# display
print(sql_query)

select user_id, profile_value from jos_user_profiles where profile_key = 'bio';


In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

In [78]:
testquery = "select user_id, profile_value from jos_user_profiles where profile_key = 'bio' and user_id = 3482;"

df_t = pd.read_sql_query(testquery, engine)
print(df_t)

   user_id                                      profile_value
0     3482  Short</br>\r\nGerhard Klimeck is an Electrical...


In [73]:
# display
display(df.head(5))
print(df.shape)

,user_id,profile_value
0,34589,Michael Brent earned a bachelor's degree in ma...
1,7922,<p>Robert Blick received his PhD from the Max-...
2,4469,<p>Takhee Lee received a B.S. and M.S. degree ...
3,2862,Mark Lundstrom is the Don and Carol Scifres Di...
4,37780,Dr. Savran received his BSME from Purdue Unive...


(8450, 2)


## Regex extract all links

In [88]:
# regex pattern match for full url
import re
pattern = re.compile(r'www[a-z.A-Z0-9+&@#\/%=~_|$?!:-]+')

In [89]:
# find all url matches - url segmentation later
urls = []
for ind,val in enumerate(df['profile_value'].to_list()):
    matches = pattern.findall(val)
    if matches == []:
        urls.append([])
    else:
        urls.append(matches)

In [90]:
print(urls)

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['www.dia2.org/', 'www.dia2.org'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['www.nitc.ac.in/nitc/user_profile/index.jsp?__tg_login=drpremrajp'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['www.ISIHighlyCited.com', 'www.esi-topics.com/nano/index.html'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['www.biancho.org/', 'www.cqc2t.org.'], [], [], [], [], [], [], [], [], [], [], 

In [91]:
# url segmentation
l_pattern = re.compile(r'linkedin[a-z.A-Z0-9+&@#\/%=~_|$?!:-]+')
t_pattern = re.compile(r'twitter[a-z.A-Z0-9+&@#\/%=~_|$?!:-]+')
f_pattern = re.compile(r'facebook[a-z.A-Z0-9+&@#\/%=~_|$?!:-]+')
#homepage - the leftovers

In [101]:
test = ['www.facebook.com/vnndaily', 'www.facebook.com/vnndaily', 'www.linkedin.com/in/vnndaily', 'www.linkedin.com/in/vnndaily']
test = ' '.join(test)
print(l_pattern.findall(test))
print(f_pattern.findall(test))
print(t_pattern.findall(test))

['linkedin.com/in/vnndaily', 'linkedin.com/in/vnndaily']
['facebook.com/vnndaily', 'facebook.com/vnndaily']
[]


In [94]:
linkedins = []
twitters = []
fbs = []
misc_url = []
for ind,val in enumerate(urls):
    t_val = ' '.join(val)
    linkedin = l_pattern.findall(t_val)
    twitter = t_pattern.findall(t_val)
    fb = f_pattern.findall(t_val)
    if len(linkedin) == 0 and len(twitter) == 0 and len(fb) == 0:
        misc_url.append(' '.join(val))
        linkedins.append(' '.join(linkedin))
        twitters.append(' '.join(twitter))
        fbs.append(' '.join(fb))
    else:
        linkedins.append(' '.join(linkedin))
        twitters.append(' '.join(twitter))
        fbs.append(' '.join(fb))
        
        #filter val with linkedin, twitter, and fb information
        for ind2,val2 in enumerate(val):
            if 'facebook' in val2 or 'twitter' in val2 or 'linkedin' in val2:
                del val[ind2]
            
        misc_url.append(' '.join(val))

In [104]:
## add to dataframe - note, there has been thus far no permutations from the default DB2 ordering
# renames: user_id to nanoHUB_user_ID__c
sf_df = pd.DataFrame()
sf_df['nanoHUB_user_ID__c'] = df['user_id']
sf_df['Twitter_profile__c'] = twitters
sf_df['Facebook_profile__c'] = fbs
sf_df['Linkedin_Bio__c'] = linkedins
sf_df['Misc_URLs__c'] = misc_url

In [105]:
display(sf_df.head(5))

,nanoHUB_user_ID__c,Twitter_profile__c,Facebook_profile__c,Linkedin_Bio__c,Misc_URLs__c
0,34589,,,,
1,7922,,,,
2,4469,,,,
3,2862,,,,
4,37780,,,,


## Send information to SF

In [106]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [107]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(sf_df)

[Success] Bulk job creation successful. Job ID = 7505w00000QipjpAAB
hello
[Success] CSV upload successful. Job ID = 7505w00000QipjpAAB
[Success] Closing job successful. Job ID = 7505w00000QipjpAAB


In [116]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000QipjpAAB',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-10-14T03:57:36.000+0000',
 'systemModstamp': '2020-10-14T03:58:59.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 8450,
 'numberRecordsFailed': 1169,
 'retries': 0,
 'totalProcessingTime': 79663,
 'apiActiveProcessingTime': 72680,
 'apexProcessingTime': 674130}

In [117]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Facebook_profile__c,Linkedin_Bio__c,Misc_URLs__c,Twitter_profile__c,nanoHUB_user_ID__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-5.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-6.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-7.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-8.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-9.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-10.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-11.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","-12.0"\n'
 '"","REQUIRED_FIELD

 '--","","","","","217464.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","217640.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","217942.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","217966.0"\n'
 '"","STRING_TOO_LONG:Misc URLs: data value too large: '
 'www.callgirlinaerocity.com/delhi-russian-escorts.html '
 'www.callgirlinaerocity.com/5star-hotel-call-girl.html '
 'www.callgirlinaerocity.com/call-girls-dehradun.html '
 'www.callgirlinaerocity.com/escorts-service-dehradun.html '
 'www.callgirlinaerocity.com/call-girl-faridabad.html '
 'www.callgirlinaerocity.com/call-girl-laxmi-nagar.html '
 'www.callgirlinaerocity.com/call-girl-in-hauzkhas.html '
 'www.callgirlinaerocity.com/call-girls-pitampura.html '
 'www.callgirlinaerocity.com/call-girls-paharganj.html '
 'www.callgirlinaerocity.com/call-girls-aerocity.html '


 '--","","","","","291193.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291260.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291287.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291296.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291365.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291385.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","www.instapaper.com/p/dienmaysakura www.plurk.com/dienmaysakura '
 'www.minds.com/dienmaysakura www.minds.com/dienmaysakura","","291375.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","","","","","291471.0"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [Las